In [1]:
#import the libraries as needed
from selenium import webdriver
from selenium import webdriver   # for webdriver
from selenium.webdriver.support.ui import WebDriverWait  # for implicit and explict waits
from selenium.webdriver.chrome.options import Options 
from selenium.webdriver.chrome.service import Service
import numpy as np
import json
from bs4 import BeautifulSoup
import requests
from tqdm import tqdm # to create loadbard in for loop
import os.path

In [2]:
# need to precise the option for date, nbr of metada for call transcript in the url paramater 
# https://en.wikipedia.org/wiki/Query_string
nbr_earning_call_metada = 200 # I think 50 is max but can easyily be adapted, with for loop or something like that
date_of_request= "2022-04-28"
url = "https://seekingalpha.com/api/v3/articles"+"?cacheBuster="\
      +str(date_of_request)\
      +"&filter[category]=earnings%3A%3Aearnings-call-transcripts&filter[since]=0"\
      +"&filter[until]=0&include=author%2CprimaryTickers%2CsecondaryTickers&isMounting=true&page[size]="\
      +str(nbr_earning_call_metada)\
      +"&page[number]=1"
url

'https://seekingalpha.com/api/v3/articles?cacheBuster=2022-04-28&filter[category]=earnings%3A%3Aearnings-call-transcripts&filter[since]=0&filter[until]=0&include=author%2CprimaryTickers%2CsecondaryTickers&isMounting=true&page[size]=200&page[number]=1'

In [3]:
# the chrome driver depend on which version of chrome you are running specify the good one
executable_path = "./chromedriver_linux"
option = webdriver.ChromeOptions()
driver = webdriver.Chrome(executable_path= executable_path,options=option)
driver.get(url)
api_json_response = driver.find_element_by_tag_name("body").text
#a quick loock of the result to check if we get results are get captcha/bot error 

print(api_json_response[0:100])
api_data_response = json.loads(api_json_response)

/home/thomas/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  after removing the cwd from sys.path.


{"data":[{"id":"4505164","type":"transcript","attributes":{"publishOn":"2022-04-29T12:05:23-04:00","


/home/thomas/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:6: DeprecationWarning: find_element_by_tag_name is deprecated. Please use find_element(by=By.TAG_NAME, value=name) instead
  


## API data response

<pre>
dict( 
    data : list(one element is one earning call metada)
    included : "don't care for now"
    meta : "don't care for now"
        )
<pre>
One earning call metadata is show below 

In [4]:
api_data_response["data"][0]

{'id': '4505164',
 'type': 'transcript',
 'attributes': {'publishOn': '2022-04-29T12:05:23-04:00',
  'isLockedPro': False,
  'commentCount': 0,
  'gettyImageUrl': None,
  'themes': {},
  'title': 'Ranger Energy Services, Inc. (RNGR) CEO Stuart Bodden on Q1 2022 Results - Earnings Call Transcript'},
 'relationships': {'author': {'data': {'id': '44211', 'type': 'author'}},
  'sentiments': {'data': []},
  'primaryTickers': {'data': [{'id': '575157', 'type': 'tag'}]},
  'secondaryTickers': {'data': []},
  'otherTags': {'data': [{'id': '96991', 'type': 'tag'},
    {'id': '17810', 'type': 'tag'},
    {'id': '49', 'type': 'tag'},
    {'id': '326', 'type': 'tag'},
    {'id': '17820', 'type': 'tag'}]}},
 'links': {'self': '/article/4505164-ranger-energy-services-inc-rngr-ceo-stuart-bodden-on-q1-2022-results-earnings-call-transcript'}}

In [5]:

id_and_path_to_earning_calls = [ (earning_call_metada["id"] ,earning_call_metada["links"]["self"]) 
                     for earning_call_metada in api_data_response["data"]]
# ""check url
id_and_path_to_earning_calls[0:2]

[('4505164',
  '/article/4505164-ranger-energy-services-inc-rngr-ceo-stuart-bodden-on-q1-2022-results-earnings-call-transcript'),
 ('4505156',
  '/article/4505156-midland-states-bancorp-inc-msbi-ceo-jeff-ludwig-on-q1-2022-earnings-call-transcript')]

## download all not yet download html from url

In [6]:
domain_name = 'https://seekingalpha.com'
need_to_download = []
for id_earning_call, path_to_earning_call in tqdm(id_and_path_to_earning_calls):
    if not os.path.isfile("data/article/"+ id_earning_call +".html"):
        # not in local directory
        need_to_download.append((id_earning_call, path_to_earning_call))
print(f"Need to download {len(need_to_download)} html file(s)")
for id_earning_call, path_to_earning_call in tqdm(need_to_download):
    with open("data/article/"+ id_earning_call+".html", "x") as file:
            url = domain_name + path_to_earning_call
            html_transcript = requests.get(url).text
            file.write(html_transcript)
    

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 50/50 [00:00<00:00, 183317.48it/s]


Need to download 7 html file(s)


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 7/7 [00:01<00:00,  3.66it/s]


## Check last file got downloaded, i.e. captcha/bot error 

In [7]:
BeautifulSoup(html_transcript).get_text()[:2000]

'L3Harris Technologies, Inc. (LHX) CEO Chris Kubasik on Q1 2022 Results - Earnings Call Transcript | Seeking Alpha                    Seeking Alpha - Go to HomepageTrendingMy PortfolioMy AuthorsTop StocksLatest NewsMarketsStock IdeasDividendsETFsEducationWatch & ListenTranscriptsIndustrialL3Harris Technologies, Inc. (LHX) CEO Chris Kubasik on Q1 2022 Results - Earnings Call TranscriptApr. 29, 2022 11:41 AM ETL3Harris Technologies, Inc. (LHX)SA Transcripts126.04K FollowersFollowL3Harris Technologies, Inc. (NYSE:LHX) Q1 2022 Earnings Conference Call April 29, 2022 8:30 AM ET Company Participants Rajeev Lalwani - VP of IR Chris Kubasik - VC and CEO Michelle Turner - CFO Conference Call Participants Robert Spingarn - Melius Research  Sheila Kahyaoglu - Jefferies  Robert Stallard - Vertical Research Partners Gautam Khanna - Cowen and Company Peter Arment - Baird  Doug Harned - Bernstein  Richard Safran - Seaport Research Partners  David Strauss - Barclays Bank  Michael Ciarmoli - Truist Sec